In [7]:
# Verificação e instalação de dependências
import sys
import subprocess
import pickle

def check_and_install_packages():
    """
    Verifica e instala pacotes necessários se houver problemas de compatibilidade
    """
    required_packages = [
        'numpy>=1.24.0',
        'pandas>=1.5.0',
        'scikit-learn>=1.3.0',
        'matplotlib>=3.7.0',
        'seaborn>=0.12.0'
    ]
    
    for package in required_packages:
        try:
            if 'numpy' in package:
                import numpy as np
                print(f"✅ NumPy version: {np.__version__}")
            elif 'pandas' in package:
                import pandas as pd
                print(f"✅ Pandas version: {pd.__version__}")
            elif 'scikit-learn' in package:
                import sklearn
                print(f"✅ Scikit-learn version: {sklearn.__version__}")
            elif 'matplotlib' in package:
                import matplotlib
                print(f"✅ Matplotlib version: {matplotlib.__version__}")
            elif 'seaborn' in package:
                import seaborn as sns
                print(f"✅ Seaborn version: {sns.__version__}")
        except ImportError as e:
            print(f"❌ Erro ao importar {package}: {e}")
            print(f"🔄 Instalando {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
    print("🎉 Todas as dependências estão funcionando corretamente!")

# Executar verificação
try:
    check_and_install_packages()
except Exception as e:
    print(f"⚠️ Problema detectado: {e}")
    print("🔄 Reinstalando NumPy especificamente...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--force-reinstall", "numpy==1.24.4"])

✅ NumPy version: 1.24.4
✅ Pandas version: 1.5.3
✅ Scikit-learn version: 1.3.0
✅ Matplotlib version: 3.7.2
✅ Seaborn version: 0.12.2
🎉 Todas as dependências estão funcionando corretamente!


# Análise de Ganho de Informação e Mutual Information

Este notebook calcula o ganho de informação e mutual information para o dataset SVM.
- **Ganho de Informação**: Mede a redução na entropia após dividir o dataset com base em um atributo
- **Mutual Information**: Mede a dependência mútua entre duas variáveis

In [11]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
import warnings
import pickle
warnings.filterwarnings('ignore')

In [2]:
#Variáveis globais
arq_dataset_pkl = '../dataset/svm.pkl'

In [ ]:
# Carregar os datasets de teste, treino e validação do arquivo pickle
print("📂 Carregando datasets preprocessados...")

try:
    import pickle
    import numpy as np
    import pandas as pd
    
    # Verificar se o arquivo existe
    import os
    if not os.path.exists(arq_dataset_pkl):
        print(f"❌ Arquivo pickle não encontrado: {arq_dataset_pkl}")
        print("🔄 Executando carregamento direto do CSV...")
        # Carregar diretamente do CSV se o pickle não existir
        raise FileNotFoundError("Pickle não encontrado")
    
    # Carregar o arquivo pickle
    with open(arq_dataset_pkl, 'rb') as f:
        datasets = pickle.load(f)
    
    # Extrair os datasets
    X_train = datasets['X_train']
    X_test = datasets['X_test']
    X_val = datasets['X_val']
    y_train = datasets['y_train']
    y_test = datasets['y_test']
    y_val = datasets['y_val']
    X_train_scaled = datasets['X_train_scaled']
    X_test_scaled = datasets['X_test_scaled']
    X_val_scaled = datasets['X_val_scaled']
    classes_mapping = datasets['classes_mapping']
    
    print("✅ Datasets carregados com sucesso!")
    print(f"   • X_train shape: {X_train.shape}")
    print(f"   • X_test shape: {X_test.shape}")
    print(f"   • X_val shape: {X_val.shape}")
    print(f"   • Classes mapping: {classes_mapping}")
    
    # Usar os dados preprocessados para análise
    print("\n📊 Usando dados preprocessados para análise de ganho de informação...")
    
    # Para o ganho de informação, vamos usar X_train e y_train
    X_for_analysis = pd.DataFrame(X_train)
    y_for_analysis = y_train
    
    print(f"   • Dados para análise: {X_for_analysis.shape}")
    print(f"   • Target para análise: {len(y_for_analysis)} amostras")

except Exception as e:
    print(f"⚠️ Erro ao carregar pickle: {e}")
    print("🔄 Carregando dados diretamente do CSV...")
    
    # Fallback: carregar diretamente do CSV
    try:
        df = pd.read_csv('../dataset/svm.csv')
        print(f"✅ Dataset CSV carregado: {df.shape}")
        
        # Assumir que a última coluna é o target
        X_for_analysis = df.iloc[:, :-1]
        y_for_analysis = df.iloc[:, -1]
        
        print(f"   • Features: {X_for_analysis.shape}")
        print(f"   • Target: {len(y_for_analysis)} amostras")
        
    except Exception as csv_error:
        print(f"❌ Erro ao carregar CSV: {csv_error}")
        print("🛑 Não foi possível carregar os dados. Verifique os arquivos.")
        raise

NameError: name 'pickle' is not defined

In [ ]:
# Carregamento do dataset
print("Carregando dataset SVM...")
df = pd.read_csv('../dataset/svm.csv')

print(f"Shape do dataset: {df.shape}")
print(f"\nPrimeiras 5 linhas:")
print(df.head())
print(f"\nInformações do dataset:")
print(df.info())
print(f"\nValores únicos por coluna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")

## Funções para Cálculo de Ganho de Informação

O ganho de informação é calculado como:
**IG(S, A) = H(S) - H(S|A)**

Onde:
- H(S) é a entropia do conjunto original
- H(S|A) é a entropia condicional após a divisão pelo atributo A

In [ ]:
def calculate_entropy(y):
    """
    Calcula a entropia de um vetor de rótulos
    """
    if len(y) == 0:
        return 0
    
    # Conta a frequência de cada classe
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    
    # Calcula a entropia
    entropy_value = -np.sum(probabilities * np.log2(probabilities + 1e-10))
    return entropy_value

def calculate_information_gain(X_feature, y):
    """
    Calcula o ganho de informação para um atributo específico
    """
    # Entropia total do conjunto
    total_entropy = calculate_entropy(y)
    
    # Valores únicos do atributo
    unique_values = np.unique(X_feature)
    
    # Entropia condicional
    weighted_entropy = 0
    for value in unique_values:
        # Índices onde o atributo tem esse valor
        indices = X_feature == value
        subset_y = y[indices]
        
        # Peso da subdivisão
        weight = len(subset_y) / len(y)
        
        # Entropia da subdivisão
        subset_entropy = calculate_entropy(subset_y)
        
        # Adiciona à entropia ponderada
        weighted_entropy += weight * subset_entropy
    
    # Ganho de informação
    information_gain = total_entropy - weighted_entropy
    return information_gain

def calculate_information_gain_ratio(X_feature, y):
    """
    Calcula a razão do ganho de informação (Information Gain Ratio)
    """
    ig = calculate_information_gain(X_feature, y)
    
    # Entropia intrínseca do atributo
    unique_values, counts = np.unique(X_feature, return_counts=True)
    probabilities = counts / len(X_feature)
    intrinsic_entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))
    
    # Evita divisão por zero
    if intrinsic_entropy == 0:
        return 0
    
    return ig / intrinsic_entropy

In [ ]:
import pickle  
# Os dados de treino foram gerados e gravados do 
#carregar os datasets do arquivo pickle
with open('datasets.pkl', 'rb') as f:
    datasets = pickle.load(f)

X_train = datasets['X_train']
y_train = datasets['y_train']
X_train_scaled = datasets['X_train_scaled']

print(datasets)

In [ ]:
# Preparação dos dados
print("Preparando os dados...")

# Identifica a coluna alvo (assumindo que é a última coluna ou uma coluna específica)
# Vamos tentar identificar automaticamente a coluna alvo
print("Colunas disponíveis:")
for i, col in enumerate(df.columns):
    print(f"{i}: {col}")

# Se não soubermos qual é a coluna alvo, vamos assumir que é a última
# Ou você pode especificar manualmente
target_column = df.columns[-1]  # Última coluna como padrão
print(f"\nUsando '{target_column}' como variável alvo")

# Separa features e target
X = df.drop(columns=[target_column])
y = df[target_column]

print(f"\nShape das features: {X.shape}")
print(f"Shape do target: {y.shape}")
print(f"Valores únicos no target: {y.nunique()}")
print(f"Distribuição do target:\n{y.value_counts()}")

## Cálculo do Ganho de Informação

Agora vamos calcular o ganho de informação para cada feature em relação à variável alvo.

In [ ]:
# Cálculo do ganho de informação para cada feature
print("Calculando ganho de informação...")

# Para features contínuas, vamos discretizá-las primeiro
def discretize_continuous_features(X, n_bins=10):
    """
    Discretiza features contínuas em bins
    """
    X_discrete = X.copy()
    
    for column in X.columns:
        if X[column].dtype in ['float64', 'int64'] and X[column].nunique() > 20:
            # Discretiza em bins
            X_discrete[column] = pd.cut(X[column], bins=n_bins, labels=False)
    
    return X_discrete

# Discretiza as features se necessário
X_discrete = discretize_continuous_features(X)

# Calcula o ganho de informação para cada feature
information_gains = {}
information_gain_ratios = {}

for column in X_discrete.columns:
    # Remove valores NaN se houver
    mask = ~(pd.isna(X_discrete[column]) | pd.isna(y))
    feature_clean = X_discrete[column][mask].values
    y_clean = y[mask].values
    
    if len(feature_clean) > 0:
        ig = calculate_information_gain(feature_clean, y_clean)
        igr = calculate_information_gain_ratio(feature_clean, y_clean)
        
        information_gains[column] = ig
        information_gain_ratios[column] = igr

# Cria DataFrame com os resultados
ig_results = pd.DataFrame({
    'Feature': list(information_gains.keys()),
    'Information_Gain': list(information_gains.values()),
    'Information_Gain_Ratio': list(information_gain_ratios.values())
})

# Ordena por ganho de informação
ig_results = ig_results.sort_values('Information_Gain', ascending=False)

print("\nGanho de Informação por Feature (Top 15):")
print(ig_results.head(15))

## Cálculo do Mutual Information

O Mutual Information mede a dependência estatística entre duas variáveis. Vamos usar a implementação do scikit-learn.

In [ ]:
# Cálculo do Mutual Information usando scikit-learn
print("Calculando Mutual Information...")

# Prepara os dados para o mutual information
# Remove valores NaN
mask = ~(X.isna().any(axis=1) | pd.isna(y))
X_clean = X[mask]
y_clean = y[mask]

# Para features categóricas, usa mutual_info_classif diretamente
# Para features contínuas, também funciona bem
try:
    # Calcula mutual information
    mi_scores = mutual_info_classif(X_clean, y_clean, random_state=42)
    
    # Cria DataFrame com os resultados
    mi_results = pd.DataFrame({
        'Feature': X_clean.columns,
        'Mutual_Information': mi_scores
    })
    
    # Ordena por mutual information
    mi_results = mi_results.sort_values('Mutual_Information', ascending=False)
    
    print("\nMutual Information por Feature (Top 15):")
    print(mi_results.head(15))
    
except Exception as e:
    print(f"Erro no cálculo do Mutual Information: {e}")
    print("Tentando com encoding das variáveis categóricas...")
    
    # Se houver erro, tenta fazer encoding das variáveis categóricas
    X_encoded = X_clean.copy()
    label_encoders = {}
    
    for column in X_encoded.columns:
        if X_encoded[column].dtype == 'object':
            le = LabelEncoder()
            X_encoded[column] = le.fit_transform(X_encoded[column].astype(str))
            label_encoders[column] = le
    
    # Tenta novamente
    mi_scores = mutual_info_classif(X_encoded, y_clean, random_state=42)
    
    mi_results = pd.DataFrame({
        'Feature': X_encoded.columns,
        'Mutual_Information': mi_scores
    })
    
    mi_results = mi_results.sort_values('Mutual_Information', ascending=False)
    
    print("\nMutual Information por Feature (Top 15):")
    print(mi_results.head(15))

## Comparação e Visualização dos Resultados

In [ ]:
# Combina os resultados para comparação
try:
    # Merge dos resultados
    combined_results = ig_results.merge(
        mi_results, 
        on='Feature', 
        how='outer', 
        suffixes=('_IG', '_MI')
    )
    
    print("Comparação entre Information Gain e Mutual Information:")
    print("="*70)
    print(combined_results.head(15))
    
    # Calcula correlação entre as métricas
    if len(combined_results) > 1:
        correlation = combined_results['Information_Gain'].corr(
            combined_results['Mutual_Information']
        )
        print(f"\nCorrelação entre Information Gain e Mutual Information: {correlation:.4f}")
    
except Exception as e:
    print(f"Erro ao combinar resultados: {e}")
    print("\nResultados separados:")
    print("\nInformation Gain:")
    if 'ig_results' in locals():
        print(ig_results.head(10))
    print("\nMutual Information:")
    if 'mi_results' in locals():
        print(mi_results.head(10))

In [ ]:
# Visualizações
plt.figure(figsize=(15, 10))

# Subplot 1: Top 15 features por Information Gain
plt.subplot(2, 2, 1)
if 'ig_results' in locals() and len(ig_results) > 0:
    top_ig = ig_results.head(15)
    plt.barh(range(len(top_ig)), top_ig['Information_Gain'])
    plt.yticks(range(len(top_ig)), top_ig['Feature'], fontsize=8)
    plt.xlabel('Information Gain')
    plt.title('Top 15 Features - Information Gain')
    plt.gca().invert_yaxis()

# Subplot 2: Top 15 features por Mutual Information
plt.subplot(2, 2, 2)
if 'mi_results' in locals() and len(mi_results) > 0:
    top_mi = mi_results.head(15)
    plt.barh(range(len(top_mi)), top_mi['Mutual_Information'])
    plt.yticks(range(len(top_mi)), top_mi['Feature'], fontsize=8)
    plt.xlabel('Mutual Information')
    plt.title('Top 15 Features - Mutual Information')
    plt.gca().invert_yaxis()

# Subplot 3: Correlação entre IG e MI (se disponível)
plt.subplot(2, 2, 3)
try:
    if 'combined_results' in locals() and len(combined_results) > 1:
        plt.scatter(combined_results['Information_Gain'], 
                   combined_results['Mutual_Information'], 
                   alpha=0.7)
        plt.xlabel('Information Gain')
        plt.ylabel('Mutual Information')
        plt.title('Information Gain vs Mutual Information')
        
        # Adiciona linha de tendência
        z = np.polyfit(combined_results['Information_Gain'].fillna(0), 
                      combined_results['Mutual_Information'].fillna(0), 1)
        p = np.poly1d(z)
        plt.plot(combined_results['Information_Gain'].fillna(0), 
                p(combined_results['Information_Gain'].fillna(0)), 
                "r--", alpha=0.8)
    else:
        plt.text(0.5, 0.5, 'Dados não disponíveis\npara correlação', 
                ha='center', va='center', transform=plt.gca().transAxes)
except Exception as e:
    plt.text(0.5, 0.5, f'Erro: {str(e)[:50]}...', 
            ha='center', va='center', transform=plt.gca().transAxes)

# Subplot 4: Distribuição dos valores
plt.subplot(2, 2, 4)
try:
    if 'ig_results' in locals() and 'mi_results' in locals():
        plt.hist(ig_results['Information_Gain'], alpha=0.7, label='Information Gain', bins=20)
        plt.hist(mi_results['Mutual_Information'], alpha=0.7, label='Mutual Information', bins=20)
        plt.xlabel('Valor')
        plt.ylabel('Frequência')
        plt.title('Distribuição dos Valores')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'Dados não disponíveis', 
                ha='center', va='center', transform=plt.gca().transAxes)
except Exception as e:
    plt.text(0.5, 0.5, f'Erro: {str(e)[:50]}...', 
            ha='center', va='center', transform=plt.gca().transAxes)

plt.tight_layout()
plt.show()

## Salvando os Resultados

In [ ]:
# Salva os resultados em arquivos CSV
try:
    # Salva Information Gain
    if 'ig_results' in locals():
        ig_results.to_csv('../dataset/information_gain_results.csv', index=False)
        print("✅ Resultados do Information Gain salvos em '../dataset/information_gain_results.csv'")
    
    # Salva Mutual Information
    if 'mi_results' in locals():
        mi_results.to_csv('../dataset/mutual_information_results.csv', index=False)
        print("✅ Resultados do Mutual Information salvos em '../dataset/mutual_information_results.csv'")
    
    # Salva resultados combinados se disponível
    if 'combined_results' in locals():
        combined_results.to_csv('../dataset/combined_information_results.csv', index=False)
        print("✅ Resultados combinados salvos em '../dataset/combined_information_results.csv'")
    
    print("\n📊 Resumo da Análise:")
    print("="*50)
    
    if 'ig_results' in locals():
        print(f"• Total de features analisadas: {len(ig_results)}")
        print(f"• Feature com maior Information Gain: {ig_results.iloc[0]['Feature']} ({ig_results.iloc[0]['Information_Gain']:.4f})")
    
    if 'mi_results' in locals():
        print(f"• Feature com maior Mutual Information: {mi_results.iloc[0]['Feature']} ({mi_results.iloc[0]['Mutual_Information']:.4f})")
    
    if 'combined_results' in locals() and len(combined_results) > 1:
        correlation = combined_results['Information_Gain'].corr(combined_results['Mutual_Information'])
        print(f"• Correlação entre IG e MI: {correlation:.4f}")
    
except Exception as e:
    print(f"❌ Erro ao salvar resultados: {e}")

print("\n🎉 Análise concluída!")

## Interpretação dos Resultados

### Information Gain vs Mutual Information

**Information Gain:**
- Mede a redução na entropia quando dividimos o dataset por um atributo
- Valores mais altos indicam maior capacidade de discriminação
- Favorece atributos com mais valores únicos (pode ter viés)

**Mutual Information:**
- Mede a dependência estatística entre variáveis
- Mais robusto para diferentes tipos de variáveis
- Não favorece atributos com mais valores únicos

### Como usar os resultados:
1. **Seleção de Features**: Use as features com maior IG ou MI para modelos de ML
2. **Feature Engineering**: Combine ou transforme features com baixo IG/MI
3. **Análise Exploratória**: Entenda quais variáveis são mais informativas
4. **Redução de Dimensionalidade**: Mantenha apenas as features mais importantes